In [1]:
import sys
assert sys.version_info >= (3, 5)
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import time
K = keras.backend
import pandas as pd
import math
import os
from sklearn.metrics import r2_score
from scipy.stats import uniform,randint
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from scipy.stats import norm
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, cophenet

tf.random.set_seed(42)
np.random.seed(42)
random_state=42

In [2]:
save_path = 'save_path'
os.chdir(save_path)

X_train_omics_unlabelled = pd.read_csv("X_train_omics_unlabelled.csv",index_col=0)
X_train_omics_labelled = pd.read_csv("X_train_omics_labelled.csv",index_col=0)
X_test_omics= pd.read_csv("X_test_omics.csv",index_col=0)
X_valid_omics= pd.read_csv("X_valid_omics.csv",index_col=0)
features = np.load("feature_selection.npy",allow_pickle=True)

train_set_labelled_y= pd.read_csv("train_set_labelled_y.csv",index_col=0)
test_set_labelled_y= pd.read_csv("test_set_labelled_y.csv",index_col=0)
valid_set_labelled_y= pd.read_csv("valid_set_labelled_y.csv",index_col=0)

X_train_omics_unlabelled = X_train_omics_unlabelled[features]
X_train_omics_labelled = X_train_omics_labelled[features]
X_test_omics = X_test_omics[features]
X_valid_omics = X_valid_omics[features]

train_set_labelled_c= pd.read_csv("train_set_labelled_c.csv",index_col=0)
train_set_unlabelled_c= pd.read_csv("train_set_unlabelled_c.csv",index_col=0)
test_set_labelled_c= pd.read_csv("test_set_labelled_c.csv",index_col=0)
valid_set_labelled_c= pd.read_csv("valid_set_labelled_c.csv",index_col=0)

#bin y 
from sklearn.preprocessing import KBinsDiscretizer
binner = KBinsDiscretizer(n_bins=10,encode="onehot-dense",strategy="uniform")
train_set_labelled_y = binner.fit_transform(train_set_labelled_y)
valid_set_labelled_y = binner.transform(valid_set_labelled_y)
test_set_labelled_y=binner.transform(test_set_labelled_y)

In [13]:
save_model_path = 'save_model_path'
os.chdir(save_model_path)

In [3]:
input_shape = X_train_omics_labelled.shape[1]

# Custom parts # 

## Helpful functions ##

In [4]:
def validation_log_lik_sampling(y_val,x_val,variational_decoder,codings_size,samples=200):
    """
    Samples a value of z for the expectation, and calculates something proportional to loglikelihood.
    
    The more samples of z, the better the MC approximation to loglik.
    
    This is how we do our evaluation on the validation and also test set. 
    
    We look at the ability to generate x given y i.e. loglik(x|y)"""
    
    x_val_len = len(x_val)
    expectation = 0
    for i in range(samples):
        z = np.random.normal(loc=0,scale=1,size=codings_size*x_val_len).reshape(x_val_len,codings_size)
        x_pred = variational_decoder([z,y_val])
        diff = (x_val-x_pred)**2
        pdf = K.sum(diff,axis=-1)
        pdf = K.exp(-pdf)
        expectation += pdf 
    expectation = expectation / samples
    lik = tf.math.log(expectation)
    lik = K.mean(lik)    
    return lik

def create_batch(x_label, y_label, x_unlabel, batch_s=32):
    '''
    Creates batches of labelled and unlabelled data. The total number of points in both batches is equal to batch_s.
    Thanks to Omer Nivron for help with this.
    
    '''
    proportion_labelled = x_label.shape[0]/(x_label.shape[0] + x_unlabel.shape[0])
    
    shape_label = x_label.shape[0]
    label_per_batch = int(np.ceil(proportion_labelled*batch_s))
    batch_idx_la = np.random.choice(list(range(shape_label)), label_per_batch)
    batch_x_la = (x_label.iloc[batch_idx_la, :])
    batch_y_la = (y_label[batch_idx_la,:])

    
    shape_unlabel = x_unlabel.shape[0]
    unlabel_per_batch = batch_s - label_per_batch
    batch_idx_un = np.random.choice(list(range(shape_unlabel)), unlabel_per_batch)
    batch_x_un = (x_unlabel.iloc[batch_idx_un, :])
    
    del batch_idx_la,batch_idx_un
            
    return batch_x_la, batch_y_la, batch_x_un


def progress_bar(iteration, total, size=30):
    """Progress bar for training"""
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    """Status bar for training"""
    metrics = " - ".join(["Loss for batch: {:.4f}".format(loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
def print_status_bar_epoch(iteration, total, training_loss_for_epoch,val_loss, metrics=None, size=30):
    """Status bar for training (end of epoch)"""
    metrics = " - ".join(
        ["trainLoss: {:.4f}  Val_loss: {:.4f} ".format(
            training_loss_for_epoch,val_loss)]
    )
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
    
def list_average(list_of_loss):
    return sum(list_of_loss)/len(list_of_loss)


def y_pred_loss(y_in):
    """Calculates loss and true y distribution given some y data.
    
    When the model calculates this it does it in batches (unlike this function which can take the whole data in).
    
    Therefore the model's learned distribution will probably not be as good as what is learnt when using the whole
    dataset. But that is one of the things that happens if we use mini-batch gradient descent.
    
    """
    y_distribution = (K.sum(y_in,axis=0) / len(y_in))
    loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y_in,y_distribution))
    return loss,y_distribution 


def rounded_accuracy(y_true,y_pred):
    """
    Calculates accuracy of classification predictions.
    
    For the 10D p vector which is y_pred, it sets the highest number to 1 and the rest to 0.
    
    It then computes accuracy.
    """
    from sklearn.metrics import accuracy_score
    b = np.zeros_like(y_pred)
    b[np.arange(len(y_pred)),y_pred.argmax(1)] = 1
    return accuracy_score(y_true,b)

## Model ##

### Custom components ###

In [5]:
class Sampling(keras.layers.Layer):
    """reparameterization trick"""
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean
     
    
    
class y_dist(keras.layers.Layer):
    """
    Custom layer that is used to learn the parameters, p, of the distribution over y.
    
    Outputs a loss and p. The loss is used for training. The loss is the categorical cross entropy loss between 
    p and every y sample. The mean of this is then taken to provide a per batch loss. 
    
    Shapes are configured for a 10D y. Change if you want to use different number of categories.
    
    """
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        
    def build(self,batch_input_shape):
        self.q = self.add_weight(name="q",shape=[1,9],initializer="uniform",trainable=True)
        super().build(batch_input_shape)
    
    def call(self,X):
        concatenated = tf.concat([self.q,tf.constant(np.array(0.0).reshape(1,-1),dtype="float32")],axis=-1)
        p = K.exp(concatenated)
        p = tf.math.divide(p,K.sum(p))
        loss = keras.losses.categorical_crossentropy(X,p)
        loss = tf.reduce_mean(loss)
        return loss,p 
    
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(10)
    
    
    
    
class FullModel(keras.models.Model):
    """
    This is the full model. This is used for training purposes.
    
    It requires an encoder, decoder, classifier and y_distribution model to be already defined (as can be done with 
    the build_model function).
    
    It returns the nloglik i.e. the loss. 
    
    This loss can then be used in gradient descent and be minimised wrt parameters (of the four component models).
    
    At test time, you will call which of the component models you want to use (as opposed to trying to "call" this 
    FullModel which you won't want to do as its purpose is just to calculate the nloglik for training).
    
    """
    def __init__(self,N_parameter,beta,variational_encoder,variational_decoder,classifier,y_distribution,**kwargs):
        super().__init__(**kwargs)
        self.encoder = variational_encoder
        self.decoder = variational_decoder
        self.classifier = classifier  
        self.y_distribution = y_distribution
        self.N = N_parameter
        self.beta = beta
    def call(self,inputs):
        """Inputs is a list, as such:
            inputs[0] is labelled X 
            inputs[1] is labelled y 
            inputs[2] is unlabelled X"""
        
        X_labelled = inputs[0]
        y_labelled = inputs[1]
        X_unlabelled = inputs[2]
        
        ############### LABELLED CASE #################
        
        codings_mean,codings_log_var,codings = self.encoder([X_labelled,y_labelled])
        y_pred_label = self.classifier(X_labelled)
        reconstructions = self.decoder([codings,y_labelled])

        #LOSSES#
        recon_loss = labelled_loss_reconstruction(codings_log_var=codings_log_var,codings_mean=codings_mean,
                                x=X_labelled,x_decoded_mean=reconstructions,beta=self.beta)
        cls_loss = labelled_cls_loss(y=y_labelled,y_pred=y_pred_label,N=self.N)
        y_dist_loss = self.y_distribution(y_labelled)[0]
        labelled_loss = recon_loss + cls_loss + y_dist_loss

        ############### UNLABELLED CASE #################
       
        y_pred_unlabel = self.classifier(X_unlabelled)
        codings_mean,codings_log_var,codings = self.encoder([X_unlabelled,y_pred_unlabel])
        reconstructions_un = self.decoder([codings,y_pred_unlabel])
        
        #LOSSES#
        
        unlabelled_recon_loss = unlabelled_loss_reconstruction(codings_log_var=codings_log_var,
                            codings_mean= codings_mean,y_pred=y_pred_unlabel,
                               x=X_unlabelled,x_decoded_mean=reconstructions_un,beta=self.beta)

        y_dist_loss = self.y_distribution(y_pred_unlabel)[0]
        unlabelled_loss = unlabelled_recon_loss + y_dist_loss
        
        ############### ALL LOSSES #######################
        
        loss = labelled_loss + unlabelled_loss
        return loss

    
    
    
    

def build_model(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50):
    
    """
    Builds deep generative model.
    
    Parameters specify the architecture. Architecture is such that encoder and decoder have same number of nodes and hidden
    layers. Done for simplicity. Classifier has its own architecture.
    
    Returns encoder,decoder,y_distribution, classifier and overall model. These can be used downstream.
    
    e.g. variational_encoder,variational_decoder,classifier,y_distribution,model = build_model(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50)
    """
       
    ########## ENCODER ###############
    
    x_in = keras.layers.Input(shape=[input_shape])
    y_in = keras.layers.Input(shape=[10])
    z = keras.layers.concatenate([x_in,y_in])
    for layer in range(n_hidden):
        z = keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal")(z)
        z = keras.layers.Dropout(0.3)(z)

    codings_mean = keras.layers.Dense(codings_size)(z)
    codings_log_var = keras.layers.Dense(codings_size)(z)
    codings = Sampling()([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(
        inputs=[x_in,y_in], outputs=[codings_mean, codings_log_var, codings])
    
    
    ########## DECODER ###############

    latent = keras.layers.Input(shape=[codings_size])
    l_merged = keras.layers.concatenate([latent,y_in])
    x = l_merged
    for layer in range(n_hidden):
        x = keras.layers.Dense(n_neurons, activation="elu",kernel_initializer="he_normal")(x)
        x = keras.layers.Dropout(0.3)(x)
    x_out = keras.layers.Dense(input_shape,activation="sigmoid")(x) 
    variational_decoder = keras.models.Model(inputs=[latent,y_in], outputs=[x_out])
    
    
    ########### CLASSIFIER ############
    
    y_classifier = x_in
    for layer in range(n_hidden_classifier):
        y_classifier = keras.layers.Dense(n_neurons_classifier, activation="elu",kernel_initializer="he_normal")(y_classifier)
        y_classifier = keras.layers.Dropout(rate=0.3)(y_classifier)
    y_pred = keras.layers.Dense(10,activation="softmax")(y_classifier) 
    classifier = keras.models.Model(inputs=[x_in], outputs=[y_pred])
    
    
    ############ Y DISTRIBUTION #############
    
    loss,p = y_dist()(y_in)
    y_distribution = keras.models.Model(inputs=[y_in],outputs=[loss,p])
    
    
    ########## FULL MODEL #############
    
    model = FullModel(N_parameter=N,beta=beta,variational_encoder=variational_encoder,
                  variational_decoder=variational_decoder,classifier=classifier,y_distribution=y_distribution)
    
    return variational_encoder,variational_decoder,classifier,y_distribution,model

### Loss functions ###

In [6]:
def custom_mse(x,x_decoded_mean):
    """returns column of squared errors. Length of column is number of samples."""
    diff = (x-x_decoded_mean)**2
    return K.sum(diff,axis=-1) /2 


def labelled_loss_reconstruction(codings_log_var,codings_mean,x, x_decoded_mean,beta=1):
    """Labelled data. This is the labelled loss."""
    recon_loss = custom_mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
    #kl loss gives vector of one value for each sample in batch 
    return K.mean(recon_loss + beta*kl_loss)

def unlabelled_loss_reconstruction(codings_log_var,codings_mean,y_pred,x,x_decoded_mean,beta=1):
    """Unlabelled data. This is the loss for unlabelled data. Reconstruction loss, KL loss and entropy
    are the included terms."""
    recon_loss = custom_mse(x,x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
    entropy = keras.losses.categorical_crossentropy(y_pred,y_pred)
    loss = K.mean(beta*kl_loss + recon_loss)
    #need to check below. We are summing over y, but we are assuming that the loss term is independent of y
    #which is not the case. Should update this for better model https://github.com/bjlkeng/sandbox/issues/3
    return K.mean(K.sum(y_pred*loss,axis=-1)) - K.mean(entropy) #note the sign


def dummy_loss(y,ypred):
    """This is a dummy loss that returns a value of zero. It is here as keras requires a loss term for each output.
        The regression_loss_for_labelled_y gives the loss which depends on the log var and mean, so we don't need another
        loss. But keras wants us to give separate losses for each. To keep keras happy, we use the dummy loss as a placeholder."""
    return 0.0

def labelled_cls_loss(y, y_pred,N=383):
    alpha = 0.1*N
    cat_xent_loss = keras.losses.categorical_crossentropy(y, y_pred)
    return alpha*K.mean(cat_xent_loss)

### Training functions ###

In [7]:
@tf.function
def train_step(inputs):
    """Decorated train_step function which applies a gradient update to the parameters"""
    with tf.GradientTape() as tape:
        loss = model(inputs,training=True)
        loss = tf.add_n([loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def fit_model(X_train_la, y_train_la, X_train_un,epochs,X_valid_la, y_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,
             valid_set=True):

    """
    Fits a single model. Gets the validation loss too if valid set exists. 
    And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Valid set is True or False depending if you have one. If you don't, the model at the end of training is saved.
    You must still pass in dummy valid sets even if valid_set=False.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    e.g. usage fit_model(X_train_omics_labelled, train_set_labelled_y, X_train_omics_unlabelled,50,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,valid_set=True)
    """
    if valid_set is True:
    
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        validation_loss = []
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))

                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la, batch_x_un= create_batch(
                        X_train_la, y_train_la, X_train_un,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),variational_decoder=variational_decoder,codings_size=codings_size)

                validation_loss.append(val_loss)
                print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                                 ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )

                #callback for early stopping
                if epoch <= patience - 1:

                    if epoch == 0:

                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")

                    else:
                        if all(val_loss<i for i in validation_loss[:-1]) is True:
                            variational_encoder.save("variational_encoder.h5")
                            variational_decoder.save("variational_decoder.h5")
                            classifier.save("classifier.h5")
                            y_distribution.save("y_distribution.h5")
                #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
                #then the statement below won't save any model, which is undesirable as we need to load a model. 

                if epoch > patience - 1:

                    latest_val_loss = validation_loss[-patience:]
                    if all(val_loss<i for i in latest_val_loss[:-2]) is True:
                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")
                    if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                        break     

        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5")     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        print("best val loss: ", min(validation_loss))
        
        return history, min(validation_loss)

        
    else:
        
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))        
        val_loss = 0

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))
                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la, batch_x_un= create_batch(
                        X_train_la, y_train_la, X_train_un,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                                 ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )
        

        variational_encoder.save("variational_encoder.h5")
        variational_decoder.save("variational_decoder.h5")
        classifier.save("classifier.h5")
        y_distribution.save("y_distribution.h5")
        
        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5")     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        
    
        return history


def fit_model_search(X_train_la, y_train_la, X_train_un,epochs,X_valid_la, y_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,
                    codings_size=50):

    """
    Use for hyperparameter search. 
    
    Fits the model. Gets the validation loss too. And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    """
    
    start = time.time()
    history = []
    
        
    @tf.function
    def train_step(inputs):
        with tf.GradientTape() as tape:
            loss = model(inputs,training=True)
            loss = tf.add_n([loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    validation_loss = []
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)

    batch_loss = []
    
    batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))

    for epoch in range(epochs):
            
            print("Epoch {}/{}".format(epoch,epochs))
            
            for i in range(batches_per_epoch):
                
                batch_x_la, batch_y_la, batch_x_un= create_batch(
                    X_train_la, y_train_la, X_train_un,batch_size)

                inputs = [batch_x_la.to_numpy(),batch_y_la,batch_x_un.to_numpy()]
                loss = train_step(inputs)

                batch_loss.append(loss)
                
                average_batch_loss = list_average(batch_loss)
                
                print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],average_batch_loss)
            
            training_loss_for_epoch = list_average(batch_loss)

            batch_loss = []
                
            history.append(training_loss_for_epoch)
            
            val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),variational_decoder=variational_decoder,codings_size=codings_size)
            
            validation_loss.append(val_loss)
            
            print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                             ,(X_train_la.shape[0] + X_train_un.shape[0]),training_loss_for_epoch,val_loss )
            
            #callback for early stopping
            
            if epoch <= patience - 1:
                
                if epoch == 0:
                
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                    
                else:
                    if all(val_loss<i for i in validation_loss[:-1]) is True:
                        variational_encoder.save("variational_encoder_intermediate.h5")
                        variational_decoder.save("variational_decoder_intermediate.h5")
                        classifier.save("classifier_intermediate.h5")
                        y_distribution.save("y_distribution_intermediate.h5")
            #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
            #then the statement below won't save any model, which is undesirable as we need to load a model. 
            
            if epoch > patience - 1:
                                
                latest_val_loss = validation_loss[-patience:]
                if all(val_loss<i for i in latest_val_loss[:-1]) is True:
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                    break     
    
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder_intermediate.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder_intermediate.h5")
    classifier = keras.models.load_model("classifier_intermediate.h5")     
    y_distribution = keras.models.load_model("y_distribution_intermediate.h5", custom_objects={
       "y_dist": y_dist
    })    
                
    done = time.time()
    elapsed = done-start
    print("Elapsed/s: ",elapsed)
    print("Final training loss: ",training_loss_for_epoch)
    print("best val loss: ", min(validation_loss))
    
    return history, min(validation_loss)

def hyperparameter_search(param_distribs,epochs,patience,n_iter,X_train_la=X_train_omics_labelled, 
                          y_train_la=train_set_labelled_y, X_train_un=X_train_omics_unlabelled,
                          X_valid_la=X_valid_omics, y_valid_la=valid_set_labelled_y):
    
    """
    Performs hyperparameter, random search. Assesses performance by determining the score on the validation set. 
    
    Saves best models (encoder, decoder and regressor) and returns these. These can then be used downstream.
    
    Also returns dictionary of the search results.
    
    Param_distribs of the form: 
            param_distribs = {
            "n_hidden": [1],
            "n_hidden_classifier": [1],
            "beta": [1],
            "n_neurons": randint.rvs(50,1000-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(49,1000-49,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(50,290-50,size=30,random_state=random_state).tolist(),
            "N" :randint.rvs().tolist(),
            "learning_rate" : ....
            #"codings_size": [50]}
            
    There must be a value for every parameter. If you know the value you want to use, set it in the param_distribs
    dictionary.
    
    Patience must be less than the number of epochs.
    
    e.g. result,variational_encoder,variational_decoder,classifier,y_distribution =
            hyperparameter_search(param_distribs,500,10,n_iter=10)

            
    """

    np.random.seed(42) #needs to be here so that everything that follows is consistent

    min_val_loss = []
    master = {}

    for i in range(n_iter): 
        K.clear_session()
        master[i] = {}
        master[i]["parameters"] = {}
        
        N= np.random.choice(param_distribs["N"])
        learning_rate= np.random.choice(param_distribs["learning_rate"])
        beta= np.random.choice(param_distribs["beta"])
        n_neurons =np.random.choice(param_distribs["n_neurons"]) 
        n_neurons_classifier =np.random.choice(param_distribs["n_neurons_classifier"]) 
        n_hidden  =np.random.choice(param_distribs["n_hidden"]) 
        n_hidden_classifier  =np.random.choice(param_distribs["n_hidden_classifier"]) 
        codings_size =np.random.choice(param_distribs["codings_size"]) 
       
        master[i]["parameters"]["N"] = N
        master[i]["parameters"]["learning_rate"] = learning_rate
        master[i]["parameters"]["beta"] = beta
        master[i]["parameters"]["n_neurons"] = n_neurons
        master[i]["parameters"]["n_neurons_classifier"] = n_neurons_classifier
        master[i]["parameters"]["n_hidden"] = n_hidden
        master[i]["parameters"]["n_hidden_classifier"] = n_hidden_classifier
        master[i]["parameters"]["codings_size"] = codings_size

        
        variational_encoder,variational_decoder,classifier,y_distribution,model = build_model(n_hidden=n_hidden,       
                                       n_neurons=n_neurons,beta=beta,n_hidden_classifier=n_hidden_classifier,
                                        n_neurons_classifier=n_neurons_classifier,N=N,codings_size=codings_size)
        
                
        history,val_loss = fit_model_search(X_train_la=X_train_la, y_train_la=y_train_la, 
                                 X_train_un=X_train_un, epochs=epochs,X_valid_la=X_valid_la, 
                                 y_valid_la=y_valid_la,patience=patience,variational_encoder=variational_encoder,
                                variational_decoder=variational_decoder, classifier=classifier,
                                y_distribution=y_distribution,model=model,Sampling=Sampling,y_dist=y_dist,
                                            batch_size=32,learning_rate=learning_rate,codings_size=codings_size)
        
        

        master[i]["val_loss"] = val_loss
        min_val_loss.append(val_loss)

        #If val loss is lowest, save this model. 
        if val_loss <=  min(min_val_loss):
            os.rename("variational_encoder_intermediate.h5","variational_encoder.h5")
            os.rename("variational_decoder_intermediate.h5","variational_decoder.h5")
            os.rename("classifier_intermediate.h5","classifier.h5")
            os.rename("y_distribution_intermediate.h5","y_distribution.h5")

        print(master)
            
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder.h5")
    classifier = keras.models.load_model("classifier.h5")     
    y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
       "y_dist": y_dist
    })    

    result = sorted(master.items(), key=lambda item: item[1]["val_loss"])
    return result,variational_encoder,variational_decoder,classifier,y_distribution


# Hyperparameter Search #

In [8]:
param_distribs = {
            "n_hidden": [1,2],
            "n_hidden_classifier": [1,2],
            "beta": [15],
            "n_neurons": randint.rvs(50,600-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(20,120-20,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(20,290-20,size=30,random_state=random_state).tolist(),
            "N" :[100,110,130,150],
            "learning_rate" : [0.001,0.0005,0.0001],
            #"codings_size": [120,60]
}

In [9]:
param_distribs = {
            "n_hidden": [1],
            "n_hidden_classifier": [1],
            "beta": [15],
            "n_neurons": [516],
           "n_neurons_classifier": [43],
            "codings_size": [208],
            "N" :[100,110,130,150],
            "learning_rate" : [0.001],}
            #"codings_size": [120,60]

In [9]:
result,variational_encoder,variational_decoder,classifier,y_distribution = hyperparameter_search(param_distribs=param_distribs,
                                                                        epochs=500,patience=10,n_iter=5)
result

Epoch 0/500

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 433.6157WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change

Epoch 0/500

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 231.8085WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change

[(3,
  {'parameters': {'N': 130,
    'beta': 15,
    'codings_size': 211,
    'learning_rate': 0.001,
    'n_hidden': 1,
    'n_hidden_classifier': 2,
    'n_neurons': 149,
    'n_neurons_classifier': 49},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=11.928445>}),
 (2,
  {'parameters': {'N': 130,
    'beta': 15,
    'codings_size': 72,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 149,
    'n_neurons_classifier': 52},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=11.945568>}),
 (0,
  {'parameters': {'N': 130,
    'beta': 15,
    'codings_size': 169,
    'learning_rate': 0.001,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 137,
    'n_neurons_classifier': 72},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=11.967422>}),
 (4,
  {'parameters': {'N': 100,
    'beta': 15,
    'codings_size': 107,
    'learning_rate': 0.0005,
    'n_hidden': 1,
    'n_hidden_classifier': 2,
    'n_neurons

Best KL model has val nloglik of 11.925144

  {'parameters': {'N': 100,
    'beta': 15,
    'codings_size': 122,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 152,
    'n_neurons_classifier': 21},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=11.928445>}),

# Single run # 

Note: we don't train on train+val set etc. as the N parameter is based on the balance between number of labelled vs unlabelled points. If you did want to train on all data, then you'd just train on everything and change the N parameter appropriately. Though you may want to check that the N is related to the ratio and not anything else.

In [21]:
variational_encoder,variational_decoder,classifier,y_distribution,model = build_model(n_hidden=2, n_neurons=152,input_shape=input_shape,beta=15,n_hidden_classifier=1,
              n_neurons_classifier=21,N=100,codings_size=122)

In [22]:
fit_model(X_train_omics_labelled, train_set_labelled_y, X_train_omics_unlabelled,100,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.0005,valid_set=True,codings_size=122)

Epoch 0/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2240/2278 [============================>.] - Loss for batch: 536.6541WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change

([<tf.Tensor: shape=(), dtype=float32, numpy=536.65405>,
  <tf.Tensor: shape=(), dtype=float32, numpy=192.84341>,
  <tf.Tensor: shape=(), dtype=float32, numpy=149.89479>,
  <tf.Tensor: shape=(), dtype=float32, numpy=124.63955>,
  <tf.Tensor: shape=(), dtype=float32, numpy=104.52094>,
  <tf.Tensor: shape=(), dtype=float32, numpy=92.40239>,
  <tf.Tensor: shape=(), dtype=float32, numpy=81.930756>,
  <tf.Tensor: shape=(), dtype=float32, numpy=74.514656>,
  <tf.Tensor: shape=(), dtype=float32, numpy=68.24049>,
  <tf.Tensor: shape=(), dtype=float32, numpy=61.030224>,
  <tf.Tensor: shape=(), dtype=float32, numpy=58.388588>,
  <tf.Tensor: shape=(), dtype=float32, numpy=53.590218>,
  <tf.Tensor: shape=(), dtype=float32, numpy=50.32064>,
  <tf.Tensor: shape=(), dtype=float32, numpy=48.151314>,
  <tf.Tensor: shape=(), dtype=float32, numpy=46.176765>,
  <tf.Tensor: shape=(), dtype=float32, numpy=43.72063>,
  <tf.Tensor: shape=(), dtype=float32, numpy=42.05734>,
  <tf.Tensor: shape=(), dtype=float3

In [23]:
test_nlog_lik = -validation_log_lik_sampling(test_set_labelled_y,X_test_omics.to_numpy(),
                                    variational_decoder=variational_decoder,codings_size=122,samples=2000)
print("test_nlog_lik = " + str(test_nlog_lik))

test_nlog_lik = tf.Tensor(12.677079, shape=(), dtype=float32)


3.9% better than just labelled data.